### Loading libs

In [6]:
### Testing simple mesh substraction

import os
import sys
import math
import gendata as gen
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from matplotlib import colors
import laspy
import cloudComPy as cc 
import psutil
from scipy import stats

print("Libs loaded")

Libs loaded


### Functions

In [7]:
def giveSpherePath(x:float = 0, y:float = 0, z:float = 0 , r:float = 10, num:int = 100, dx:float = 0.1, h:float = 5):
    """give_sphere Create a sphere point cloud
    Parameters"""
    dataDir = os.path.join(os.environ["USERPROFILE"], "CloudComPy/data")
    print("dataDir: ", dataDir)
    if not os.path.exists(dataDir):
        os.makedirs(dataDir)
    # points = []
    # for i in range(0, num):
    #     factor = stats.norm.pdf((np.arange(-5, 5, 0.001)), 0, 1)
    #     # factor = normedgauss()        # A value between 0 and 1 following a gaussian
    #     ir = r * factor
    #     itheta = np.arccos(np.random.uniform(-1, 1))
    #     iphi = np.random.uniform(0, 2 * np.pi)
    #     ix = x + ir * np.sin(itheta) * np.cos(iphi)
    #     iy = y + ir * np.sin(itheta) * np.sin(iphi)
    #     iz = z + ir * np.cos(itheta)
    #     points.append((ix, iy, iz))

    dataName = "dataSample_%s.xyz" % h
    if dx != 0:
        dataName = "dataSample_%s_%s.xyz" % (h,dx)
    sample = os.path.join(dataDir, dataName)
    vx = np.array([])
    vy = np.array([])
    if not os.path.exists(sample):
        print("Generating data: start X and Y!")
        for i in range(0, num):
            factor = stats.norm.pdf((np.arange(-5, 5, 0.001)), 0, 1)
            ir = r * factor
            itheta = np.arccos(np.random.uniform(-1, 1))
            iphi = np.random.uniform(0, 2 * np.pi)
            vx = np.append(arr = vx, values =  x + ir * np.sin(itheta) * np.cos(iphi))
            vy = np.append(arr = vy, values = y + ir * np.sin(itheta) * np.sin(iphi))
        print("Generating data: X and Y done!")

        with open(sample, 'w') as f:
            for xCoord in vx:
                for yCoord in vy:
                    zCoord = math.sin(h * math.sqrt(xCoord**2 + yCoord**2)) / math.sqrt(xCoord**2 + yCoord**2)
                    f.write("%f %f %f\n" % (xCoord, yCoord, zCoord))
                print("Generating data: Z done!")
    return sample

### Checking paths

In [8]:
path1 = r"C:\Users\szinp\Desktop\CloudComPy39_20220513\CloudCompPy_Computing-CloudPoint\\testSphere_1.txt"
path2 = r"C:\Users\szinp\Desktop\CloudComPy39_20220513\CloudCompPy_Computing-CloudPoint\\testSphere_2.txt"

assert os.path.isfile(path1)
assert os.path.isfile(path2)
print("Paths are correct")

Paths are correct


### Loading data into CloudComPy

In [9]:
# os.environ["_CCTRACE_"]="ON"                                           # only if you want debug traces from C++

gen.createSymbolicLinks() # required for tests on build, before cc.initCC.init
cc.initCC()
#TODO find a way to load a las file into this cloudComPy
cloud1 = cc.loadPointCloud(path1)
cloud2 = cc.loadPointCloud(path2)
# cloud = cc.loadPointCloud(giveSpherePath(num=5))
#TODO Slice the point cloud???

### Creating the mesh form cloud point data

In [10]:
if cc.isPluginMeshBoolean():
    import cloudComPy.MeshBoolean
    # cc.MeshBoolean.computeMeshBoolean(...)

# meshFromCloud1 = cc.ccMesh.triangulate(cloud = cloud1, type = cc.TRIANGULATION_TYPES.DELAUNAY_2D_BEST_LS_PLANE )
# meshFromCloud2 = cc.ccMesh.triangulate(cloud = cloud2, type = cc.TRIANGULATION_TYPES.DELAUNAY_2D_BEST_LS_PLANE )
meshFromCloud1 = cc.ccMesh.triangulate(cloud = cloud1, type = cc.TRIANGULATION_TYPES.DELAUNAY_2D_AXIS_ALIGNED, updateNormals = True ,  dim =2)
meshFromCloud2 = cc.ccMesh.triangulate(cloud = cloud2, type = cc.TRIANGULATION_TYPES.DELAUNAY_2D_AXIS_ALIGNED, updateNormals = True ,  dim = 2 )

#? cc.RasterizeToMesh(...) might be useful for rasterizing a cloud to a mesh

print(type(meshFromCloud1))
print(type(meshFromCloud2))


<class '_cloudComPy.ccMesh'>
<class '_cloudComPy.ccMesh'>


### Creating a boolean intersection of meshes

In [ ]:
# DIFF= cloudComPy.MeshBoolean._MeshBoolean.CSG_OPERATION.DIFF
# INTERSEC= cloudComPy.MeshBoolean._MeshBoolean.CSG_OPERATION.INTERSECT
# SYM_DIFF= cloudComPy.MeshBoolean._MeshBoolean.CSG_OPERATION.SYM_DIFF
# UNION= cloudComPy.MeshBoolean._MeshBoolean.CSG_OPERATION.UNION

# names= {'DIFF': cloudComPy.MeshBoolean._MeshBoolean.CSG_OPERATION.DIFF,
# 'INTERSECT': cloudComPy.MeshBoolean._MeshBoolean.CSG_OPERATION.INTERSECT,
#  'SYM_DIFF': cloudComPy.MeshBoolean._MeshBoolean.CSG_OPERATION.SYM_DIFF, 
#  'UNION': cloudComPy.MeshBoolean._MeshBoolean.CSG_OPERATION.UNION}
 
# values= {0: cloudComPy.MeshBoolean._MeshBoolean.CSG_OPERATION.UNION, 
# 1: cloudComPy.MeshBoolean._MeshBoolean.CSG_OPERATION.INTERSECT, 
# 2: cloudComPy.MeshBoolean._MeshBoolean.CSG_OPERATION.DIFF, 
# 3: cloudComPy.MeshBoolean._MeshBoolean.CSG_OPERATION.SYM_DIFF}

In [12]:
operationType = cc.MeshBoolean.CSG_OPERATION.INTERSECT
# print(operationType.INTERSECT)
# print(operationType.as_integer_ratio)
# print(operationType.name)
# print(type(operationType))
#? Does not work with large meshes
#? cc.MeshBoolean.computeMeshBoolean(...) since it does not work properly will try to use ANOTHER libary - PyMesh for that matter

cloudIntersectionMesh = cc.MeshBoolean.computeMeshBoolean( meshFromCloud1,  meshFromCloud2,  operationType)
# mesh = cc.MeshBoolean.computeMeshBoolean(sphere, cylinder, cc.MeshBoolean.CSG_OPERATION.INTERSECT)

# # cloudIntersectionMesh = cc.MeshBoolean.computeMeshBoolean(meshFromCloud1, meshFromCloud2,cc.MeshBoolean.CSG_OPERATION.DIFF)
print(type(cloudIntersectionMesh))
print(cloudIntersectionMesh)

<class 'NoneType'>
None


### Gathering points/verts data from final mesh

In [ ]:
#TODO
# pointCloudOfIntersection = cloudIntersectionMesh.getAssociatedCloud()

### Saving the results

In [17]:
# cloudIntersection.setName("cloudIntersection")

meshFromCloud1.setName("cloud1")
meshFromCloud2.setName("cloud2")
print(cc.SaveMesh(meshFromCloud1, "cloud1.ply"))
print(cc.SaveMesh(meshFromCloud2, "cloud2.ply"))
# cloudIntersection = cc.subtractClouds(cloud1, cloud2)
# cloudIntersection.save("testSubstract.las")

CC_FERR_NO_ERROR
CC_FERR_NO_ERROR


### Creating primitives

In [ ]:
# # ccGLMatrix() is a transformation matrix?
# tr1 = cc.ccGLMatrix()
# tr1.initFromParameters(0.1, 0.2, 0.3, (8.0, 0.0, 0.0))
# box = cc.ccBox((1., 2., 3.), tr1, "aBox")
# if box.getName() != 'aBox':
#     raise RuntimeError
# if box.size() != 12:
#     raise RuntimeError

# tr5 = cc.ccGLMatrix()
# tr5.initFromParameters(0.0, (0., 0., 0.), (-7.0, 5.0, 1.0))
# sphere = cc.ccSphere(1.5, tr5, "aSphere", 72)
# if sphere.getName() != 'aSphere':
#     raise RuntimeError
# if sphere.size() != 10224:
#     raise RuntimeError
# #! The IP is 192.168.2.99
# #! User:user
# #! PSW: gtergvi

### Computing approximate Cloud distance to mesh
[]: # Language: python
[]: # Path: testingFunctions.ipynb

In [ ]:
# stats = cc.DistanceComputationTools.computeApproxCloud2MeshDistance(cloud, sphere)
# print(stats) # min, max, mean, variance, error max
# nbCpu = psutil.cpu_count()
# bestOctreeLevel = cc.DistanceComputationTools.determineBestOctreeLevel(cloud,box)
# params = cc.Cloud2MeshDistancesComputationParams()
# params.maxThreadCount = nbCpu
# params.octreeLevel = bestOctreeLevel
# cc.DistanceComputationTools.computeCloud2MeshDistances(cloud, box, params)
# bestOctreeLevel = cc.DistanceComputationTools.determineBestOctreeLevel(box.getAssociatedCloud(), sphere)
# params.octreeLevel = bestOctreeLevel
# cc.DistanceComputationTools.computeCloud2MeshDistances(sphere.getAssociatedCloud(), sphere, params)

[5.562787055969238, 9.822517395019531, 7.639709949493408, 0.4474430978298187, 0.030983304604887962]


1

In [ ]:
# cc.SaveEntities([cloud, box, sphere], os.path.join("", "entities2.bin"))

_cloudComPy.CC_FILE_ERROR.CC_FERR_NO_ERROR

In [16]:
import os
import sys
import math

os.environ["_CCTRACE_"]="ON" # only if you want C++ debug traces

from gendata import dataDir

import cloudComPy as cc

tr1 = cc.ccGLMatrix()
tr1.initFromParameters(0.0, (0., 0., 0.), (1.0, 0.0, 0.0))
sphere = cc.ccSphere(2, tr1, "aSphere")

cylinder = cc.ccCylinder(2.0, 5.0)

if cc.isPluginMeshBoolean():
    import cloudComPy.MeshBoolean
    mesh = cc.MeshBoolean.computeMeshBoolean(sphere, cylinder, cc.MeshBoolean.CSG_OPERATION.INTERSECT)
    if mesh.size() != 1020:
        raise RuntimeError

    cc.SaveEntities([sphere, cylinder, mesh], "MeshBoolean.bin")